# Conversational Retrieval Chain

Date: 2024/08/12

In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
# RecursiveCharacterTextSplitterは文字数でテキスト分割。
# SpacyTextSplitterはトークンでテキスト分割。

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# FAISSへembeddingsを保持
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector_store = FAISS.from_documents(documents, embeddings)

In [24]:
# LLM

from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [25]:
# Promptのテンプレート

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [31]:
# 履歴を見たRetrieval の Chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)